In [1]:
import cv2
import numpy as np
import os

## KNN Code

In [2]:
def dist(x1, x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X, Y, queryPt, k = 5):
    
    vals = []
    m = X.shape[0]

    for i in range(m):
        d = dist(queryPt, X[i])
        vals.append((d,Y[i]))

    vals = sorted(vals)

    vals = vals[:k]
    vals = np.array(vals)

    vals = np.unique(vals[:1],return_counts=True)
    index = vals[1].argmax()

    return  vals[0][index]



## Data Preparation

In [5]:
datasetPath = "./data/"
faceData = []
labels = []

class_id = 0
names = {}

for fx in os.listdir(datasetPath):
    if(fx.endswith('.npy')):
        print("Loaded "+fx)
        names[class_id] = fx[:-4]
        data_item = np.load(datasetPath+fx)
        faceData.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))
        labels.append(target)
        class_id += 1
        print(data_item.shape, target.shape)

faceDataset = np.concatenate(faceData, axis=0)
faceLabels = np.concatenate(labels, axis=0).reshape(-1,1)

print(faceDataset.shape,faceLabels.shape)

trainSet = np.concatenate((faceDataset,faceLabels), axis=1)

Loaded Barack Obama.npy
(30, 30000) (30,)
Loaded Bill Gates.npy
(30, 30000) (30,)
Loaded Donald Trump.npy
(30, 30000) (30,)
Loaded Elon Musk.npy
(30, 30000) (30,)
Loaded Jack Ma.npy
(30, 30000) (30,)
Loaded Jeff Bezos.npy
(30, 30000) (30,)
Loaded Mark Zuckerberg.npy
(30, 30000) (30,)
Loaded Modi.npy
(30, 30000) (30,)
Loaded Mukesh Ambani.npy
(30, 30000) (30,)
Loaded Naman.npy
(50, 30000) (50,)
Loaded Satya Nadella.npy
(30, 30000) (30,)
Loaded Steve Jobs.npy
(30, 30000) (30,)
Loaded Sundar Pichai.npy
(30, 30000) (30,)
Loaded Tom Cruise.npy
(30, 30000) (30,)
Loaded Xi Jinping.npy
(30, 30000) (30,)
(470, 30000) (470, 1)


## Read Video Stream and Extract Faces

In [6]:
def detectFace(img, CascadeClassifier, scaleFactor, minNeighbours):
    faces = CascadeClassifier.detectMultiScale(img, scaleFactor, minNeighbours)
    color = (255,0,0)

    for face in faces:
        x,y,w,h = face
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0),1)

        # Extract ROI
        offset = 10
        roi = img[y-offset : y+h+offset, x-offset : x+w+offset]
        roi = cv2.resize(roi, (100,100))

        out = knn(faceDataset, faceLabels, roi.flatten())
        predName = names[int(out)]
        cv2.putText(img, predName, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
    return img

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

scaleFactor = 1.1
minNeighbours = 10

test = "./Test/"
images = os.listdir(test)
image = images[0]
img = cv2.imread(test+image)
img = detectFace(img, face_cascade, scaleFactor, minNeighbours)
cv2.imshow("Image", img)
if cv2.waitKey(1) & 0xff == ord('q'):
    cv2.destroyAllWindows()

'./Test/1.jpg'